In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

Будем работать с набором данных по качеству белого вина (репозиторий UCI). Загружаем данные.

In [2]:
data = pd.read_csv('../data/winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.2 KB


Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.

In [16]:
y = data['quality']
data1 = data.drop(columns='quality')

X_train, X_holdout, y_train, y_holdout = train_test_split(data1, y, test_size=0.3,random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

# Линейная регрессия

Обучите простую линейную регрессию.

In [17]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

#### Вопрос 1: Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?

In [30]:
linreg_pred = linreg.predict(X_holdout_scaled)
linreg_train = linreg.predict(X_train_scaled)

In [31]:
mse_train = mean_squared_error(y_train, linreg_train)
mse_test = mean_squared_error(y_holdout, linreg_pred)

print("Mean squared error (train): %.3f" % mse_train)
print("Mean squared error (test): %.3f" % mse_test)


Mean squared error (train): 0.558
Mean squared error (test): 0.584


#### Ответ(1): 0.558 и 0.584

#### Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.
#### Вопрос 2: Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?

In [39]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
np.abs(linreg.coef_)

array([9.78219223e-02, 1.92259947e-01, 1.83224449e-04, 5.38164096e-01,
       8.12724353e-03, 4.21804406e-02, 1.43040227e-02, 6.65720472e-01,
       1.50036006e-01, 6.20533605e-02, 1.29533447e-01])

In [40]:
pd.DataFrame({'feat': X_train.columns,
              'coef': np.abs(linreg.coef_).flatten().tolist()}).sort_values(by='coef', ascending=False)

,feat,coef
7,density,0.665720
3,residual sugar,0.538164
1,volatile acidity,0.192260
8,pH,0.150036
10,alcohol,0.129533
0,fixed acidity,0.097822
9,sulphates,0.062053
5,free sulfur dioxide,0.042180
6,total sulfur dioxide,0.014304
4,chlorides,0.008127


#### Ответ(2): density

# Lasso-регрессия

Обучите Lasso-регрессию с небольшим коэффициентом α=0.01 (слабая регуляризация). Пусть опять random_state=17.

In [42]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)

Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?

In [45]:
pd.DataFrame({'feat': X_train.columns,
              'coef': np.abs(lasso1.coef_).flatten().tolist()}).sort_values(by='coef', ascending=False)

,feat,coef
10,alcohol,0.322425
3,residual sugar,0.256363
7,density,0.235492
1,volatile acidity,0.188479
8,pH,0.067277
5,free sulfur dioxide,0.043088
9,sulphates,0.029722
4,chlorides,0.002747
0,fixed acidity,0.000000
2,citric acid,0.000000


Теперь определите лучшее значение α
в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.

In [46]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv='warn', eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [51]:
pd.DataFrame({'feat': X_train.columns,
              'coef': np.abs(lasso_cv.coef_).flatten().tolist()}).sort_values(by='coef', ascending=False)

,feat,coef
7,density,0.642524
3,residual sugar,0.523266
1,volatile acidity,0.191992
8,pH,0.145426
10,alcohol,0.139555
0,fixed acidity,0.091858
9,sulphates,0.060585
5,free sulfur dioxide,0.042865
6,total sulfur dioxide,0.012546
4,chlorides,0.006558


#### Ответ(3): citric acid

#### Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.
#### Вопрос 4: Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?

In [54]:
lasso_cv_pred = lasso_cv.predict(X_holdout_scaled)
lasso_cv_train = lasso_cv.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, lasso_cv_train)
mse_test = mean_squared_error(y_holdout, lasso_cv_pred)

print("Mean squared error (train): %.3f" % mse_train)
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (train): 0.558
Mean squared error (test): 0.583


#### Ответ(4): 0.558 и 0.583

# Случайный лес

Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.

In [89]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

#### Вопрос 5: Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?

In [60]:
forest_params = {'max_depth': list(range(10, 25)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

In [68]:
forest_grid = GridSearchCV(forest, forest_params, scoring='neg_mean_squared_error')
forest_grid.fit(X_train_scaled, y_train)
forest_grid_train = forest_grid.predict(X_train_scaled)

In [72]:
forest_grid.best_params_, forest_grid.best_score_

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 -0.45285564564445485)

In [91]:
forest_pred = forest.predict(X_holdout_scaled)
forest_train = forest.predict(X_train_scaled)

In [92]:
#mse_train_grid = mean_squared_error(y_train, forest_grid_train)
mse_train = mean_squared_error(y_train, forest_train)
mse_test = mean_squared_error(y_holdout, forest_pred)

In [71]:
сv_scores = cross_val_score(forest_grid, X_train_scaled, y_train, 
                            scoring='neg_mean_squared_error', n_jobs=1)

In [74]:
сv_scores, сv_scores.mean() #ДЛЯ САМОГО ЛУЧШЕГО ЭТО

(array([-0.53054638, -0.44847122, -0.45828223]), -0.47909994473242595)

In [93]:
print("Mean squared error (train): %.3f" % mse_train)
print("Mean squared error (cv): %.3f" % сv_scores.mean())
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (train): 0.075
Mean squared error (cv): -0.479
Mean squared error (test): 0.422


#### Ответ(5): 0.075, 0.46 и 0.421

#### К сожалению, результаты GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании random_state). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).
#### Вопрос 6: Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?

In [78]:
forest = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=None, oob_score=False, random_state=17,
           verbose=0, warm_start=False)

In [80]:
cv_scores = cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error', n_jobs=-1) 

In [81]:
cv_scores, cv_scores.mean() #ДЛЯ САМОГО ЛУЧШЕГО ЭТО

(array([-0.48446363, -0.43173778, -0.45338141]), -0.45652760412096344)

In [82]:
forest_pred = forest.predict(X_holdout_scaled)
forest_train = forest.predict(X_train_scaled)
mse_train = mean_squared_error(y_train, forest_train)
mse_test = mean_squared_error(y_holdout, forest_pred)
mse_train, mse_test

(0.08356267896068717, 0.4097756677231903)

In [83]:
print("Mean squared error (cv): %.3f" % cv_scores.mean())
print("Mean squared error (test): %.3f" % mse_test)

Mean squared error (cv): -0.457
Mean squared error (test): 0.410


#### Ответ(6): 0.457 и 0.41

#### Оцените важность признаков с помощью случайного леса.
#### Вопрос 7: Какой признак оказался главным в настроенной модели случайного леса?

In [88]:
pd.DataFrame({'feat': X_train.columns,
              'coef': np.abs(forest.feature_importances_).flatten().tolist()}).sort_values(by='coef', ascending=False)

,feat,coef
10,alcohol,0.224432
1,volatile acidity,0.119393
5,free sulfur dioxide,0.116147
8,pH,0.072806
6,total sulfur dioxide,0.071318
3,residual sugar,0.070160
7,density,0.069367
4,chlorides,0.067982
0,fixed acidity,0.064268
2,citric acid,0.062945


#### Ответ(7): Alcohol